# new model

torch 모델을 다른것을 사용해서 확인하기에, 새로운 그리드 파일을 연것.

encoder, transformer 이미지를 테스트 예정.

encoder 이미지는 1개라서 그리드는 필요없음.

---
그리고 데이터 변경

In [35]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
from torchvision import transforms, datasets
import pandas as pd
import os
from glob import glob
import torchvision.models as models
import sys
import math



p = os.path.abspath('../') # 상위 폴더를 사용하기 위해서.
sys.path.insert(1, p)
from pytorchtools.pytorchtools import EarlyStopping # 상위 폴더에 추가된 모듈.

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  1.10.2  Device :  cuda


In [36]:
import numpy as np
import librosa, librosa.display 
import matplotlib.pyplot as plt
#window sizde : FFT를 할때 참조할 그래프 길이 ( 프레임 하나당 sample 수 )
#자연어 처리에서는 25ms 사용. https://ahnjg.tistory.com/93
#초당 50000hz 중 1250개씩 윈도우 사이즈로 사용.
sr=16000
win_length =  np.int64(sr/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.

# data 분류

In [37]:
#1. train, test 나누기

import os
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split # train , test 분리에 사용.


pathology = glob('../../voice_data/organics/pathology/phrase/*.wav')
healthy = glob('../../voice_data/organics/healthy/phrase/*.wav')
print("Pathology : ",len(pathology))
print("Healthy: ",len(healthy))


pathology= [ path.split("\\")[-1] for path in pathology] # path 데이터 변환.
healthy= [ path.split("\\")[-1] for path in healthy] # path 데이터 변환.
 # path 데이터 변환


X = pathology+healthy # path 데이터 합
print("총 데이터수 : ",len(X))
Y = [] # 라벨
for idx,x in enumerate(X):
    if idx<597:
        Y.append("pathology")
    else:
        Y.append("healthy")

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y, random_state=456)
#stratify를 넣어서, test에도 라벨별 잘 분류되게 한다.

print("---")
print("훈련 셋 : ",len(Y),Counter(Y))
print("테스트 셋 : ",len(Y_test),Counter(Y_test))
print("---")

Pathology :  597
Healthy:  634
총 데이터수 :  1231
---
훈련 셋 :  984 Counter({'healthy': 507, 'pathology': 477})
테스트 셋 :  247 Counter({'healthy': 127, 'pathology': 120})
---


In [38]:
#1. train, test 나누기
#stratified kfold
import os
import random #데이터 shuffle 사용
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter



skf_iris = StratifiedKFold(n_splits=5,shuffle=True,random_state=456)
cnt_iter = 0

X_train_list = [] #데이터 셋 보관
Y_train_list = []

X_valid_list = []
Y_valid_list = []

for train_idx, test_idx in skf_iris.split(X,Y):
    
    #split으로 반환된 인덱스를 이용하여, 학습 검증용 테스트 데이터 추출
    cnt_iter += 1
    X_train, X_valid = [X[idx] for idx in train_idx.tolist() ], [X[idx] for idx in test_idx.tolist() ]
    Y_train, Y_valid = [Y[idx] for idx in train_idx.tolist() ], [Y[idx] for idx in test_idx.tolist() ]
    
    X_train_list.append(X_train)
    X_valid_list.append(X_valid)
    
    Y_train_list.append(Y_train)
    Y_valid_list.append(Y_valid)
    
    
    #학습 및 예측
    
    label_train = Y_train
    label_test = Y_valid
    unique_train, train_counts = np.unique(label_train, return_counts = True)
    unique_test, test_counts = np.unique(label_test, return_counts = True)
    
    uniq_cnt_train = dict(zip(unique_train, train_counts))
    uniq_cnt_test = dict(zip(unique_test, test_counts))
    
    
    
    print('교차 검증 : {}'.format(cnt_iter))
    print('학습 레이블 데이터 분포 : \n', uniq_cnt_train)
    print('검증 레이블 데이터 분포 : \n', uniq_cnt_test,'\n')



교차 검증 : 1
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 381}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 96} 

교차 검증 : 2
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 381}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 96} 

교차 검증 : 3
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 382}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 95} 

교차 검증 : 4
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 382}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 95} 

교차 검증 : 5
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 382}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 95} 



In [39]:
#2. random over sampling
for i in range(5):
    X_temp = np.array(X_train_list[i]).reshape(-1,1)#각 데이터를 다 행으로 넣음. (1194,1)
    #Y = np.array(Y)
    ros = RandomOverSampler(random_state = 123)
    X_res,Y_res = ros.fit_resample(X_temp,Y_train_list[i])
    
    print("\n fold{} ".format(i))
    print('before dataset shape {}'.format(Counter(Y_train_list[i])) )
    print('Resampled dataset shape {}'.format(Counter(Y_res)) )
    
    #원래대로 돌리기
    X_res=X_res.reshape(1, -1)
    X_train_list[i]=X_res[0].tolist()
    Y_train_list[i]=Y_res




 fold0 
before dataset shape Counter({'healthy': 406, 'pathology': 381})
Resampled dataset shape Counter({'pathology': 406, 'healthy': 406})

 fold1 
before dataset shape Counter({'healthy': 406, 'pathology': 381})
Resampled dataset shape Counter({'healthy': 406, 'pathology': 406})

 fold2 
before dataset shape Counter({'healthy': 405, 'pathology': 382})
Resampled dataset shape Counter({'pathology': 405, 'healthy': 405})

 fold3 
before dataset shape Counter({'healthy': 405, 'pathology': 382})
Resampled dataset shape Counter({'pathology': 405, 'healthy': 405})

 fold4 
before dataset shape Counter({'healthy': 406, 'pathology': 382})
Resampled dataset shape Counter({'pathology': 406, 'healthy': 406})


In [40]:
import pickle

 
#load
with open("../../voice_data/organics/phrase_sig_dict.pickle","rb") as fr:
    phrase_dict = pickle.load(fr)

    

In [41]:
phrase_dict['1040-phrase.wav'].shape

(36608,)

# data set

In [42]:
from torch.utils.data import Dataset, DataLoader

classes = ["pathology","healthy"]
sr=16000
win_length =  np.int64(sr/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



class svd_dataset(Dataset):
    def __init__(self,data_path_list,classes,data_num,training):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list[data_num]
        self.data_num = data_num
        self.training = training
        self.label = svd_dataset.get_label(self.path_list,training,data_num)
        self.classes=classes
        
    
    @classmethod
    def get_label(cls,data_path_list,training,data_num):
        label_list=[]
        
        if training:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_train_list[data_num][idx])
        else:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_valid_list[data_num][idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다.     
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        sig_length = sig.size #sig length 리턴에 사용. feature 추출및 학습에 사용.
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        length = 101951
        sig=pad1d(sig,length)
        sig=torch.from_numpy(sig).type(torch.float32)# 타입 변화
        
        return sig, self.classes.index(self.label[idx]),sig_length

In [43]:
# test set 제작을 위한 class
class svd_test_set(Dataset):
    def __init__(self,data_path_list,classes):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list
        self.label = svd_test_set.get_label(self.path_list)
        self.classes=classes

        
    
    @classmethod
    def get_label(cls,data_path_list):
        label_list=[]
        
        for idx,x in enumerate(data_path_list):
            label_list.append(Y_test[idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다. 
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        sig_length = sig.size #sig length 리턴에 사용. feature 추출및 학습에 사용.
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        length = 101951
        sig=pad1d(sig,length)
        sig=torch.from_numpy(sig).type(torch.float32)# 타입 변화
        
        return sig, self.classes.index(self.label[idx]),sig_length

# data loader

In [44]:
#3. 하이퍼 파라미터
BATCH_SIZE =  32 #한 배치당 32개 음성데이터
EPOCHS = 50 # 전체 데이터 셋을 50번 반복

In [45]:
#DATA LOADER 함수가 BATCH_size 단위로 분리해 지정.

#확인을 위해 데이터셋 하나만 확인



train_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_train_list,
                                                   classes,
                                               data_num=0,
                                               training=True
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           ) # 순서가 암기되는것을 막기위해.

validation_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_valid_list,
                                               classes,
                                               data_num=0,
                                               training=False
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,) 



In [46]:
# 테스트 데이터 로더.

test_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_test_set(
                                                   X_test,
                                                   classes,
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 

# model

https://pytorch.org/audio/main/tutorials/speech_recognition_pipeline_tutorial.html


dat1=torch.randn((32,12,318,768))
model1=nn.Conv2d(12,3,(3,3))
dat2=model1(dat1)
print(dat2.size())
#torch.Size([32, 3, 316, 766])
res_18_sample = models.resnet18(pretrained=True)
dat3=res_18_sample(dat2)
print(dat3.size())



----

In [47]:
###
sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.


        
class wav_path_Net(nn.Module):
    def __init__(self,):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        
        self.padded_frame=318
        
        input_dim =101951
        
        self.fc = nn.Sequential(nn.Linear(24*1024,1024),
                                nn.BatchNorm1d(1024),
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(1024,256),
                                nn.BatchNorm1d(256),
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(256,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=[ feat.mean(axis=1) for feat  in  hidden_feature]
        return hidden_feature
        
        
    def forward(self,x,sig_size):
        x_trans,trans_len = self.model_wav2vec.extract_features(x,sig_size)
        self.model_wav2vec.extract_features(x_train,sig_size)
        x = self.merge(x)
        x = torch.stack(x,dim=1)
        x = x.view(-1,24*1024)
        x= self.fc(x)
    
        return x 
    


In [52]:
###
sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



        
class wav_path_Net(nn.Module):
    def __init__(self,num):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        #self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        self.num = num        
        #input_dim =101951
        
        self.padded_frame=318
        
        self.fc = nn.Sequential(
                                nn.Linear(1024,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=hidden_feature.mean(axis=1)
        return hidden_feature
    
    def zero_padding(self,hidden_feature,seq_len):
        
        zero_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            zero_padding_array[i:,seq_len[i]:,:]=0
            
        hidden_feature=hidden_feature*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        return hidden_feature
        
    def forward(self,x,sig_size):
        with torch.inference_mode():
            trans_x,trans_len = self.model_wav2vec2.extract_features(x,sig_size)
            encoder_x,encoder_len = self.model_wav2vec2.feature_extractor(x,sig_size)
        if(self.num == 24):
            x = encoder_x
            x = self.zero_padding(x,encoder_len)
        else:
            x = trans_x[self.num]
            x = self.zero_padding(x,trans_len)
        x = self.merge(x)
        #x = torch.stack(x,dim=1)
        #x = x.view(-1,1024)
        x= self.fc(x)
    
        return x 
    


In [49]:

#mean 변경 및 데이터 변경 실험
        
class wav_path_Net(nn.Module):
    def __init__(self,num):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        #self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        self.num = num
        
        #input_dim =101951
        
        self.padded_frame=318
        
        self.fc = nn.Sequential(
                                nn.Linear(1024,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=hidden_feature.sum(axis=1)/(hidden_feature!=0).sum(axis=1)### 수정 zero 포함않게.
        return hidden_feature
    
    def zero_padding(self,hidden_feature,seq_len):
        
        zero_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            zero_padding_array[i:,seq_len[i]:,:]=0
            
        hidden_feature=hidden_feature*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        return hidden_feature
        
    def forward(self,x,sig_size):
        with torch.inference_mode():
            trans_x,trans_len = self.model_wav2vec2.extract_features(x,sig_size)
            encoder_x,encoder_len = self.model_wav2vec2.feature_extractor(x,sig_size)
        if(self.num == 24):
            x = encoder_x
            x = self.zero_padding(x,encoder_len)
        else:
            x = trans_x[self.num]
            x = self.zero_padding(x,trans_len)
        x = self.merge(x)
        #x = torch.stack(x,dim=1)
        #x = x.view(-1,1024)
        x= self.fc(x)
    
        return x 
    


In [50]:
###
# model for enocoder

sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



        
class wav_path_Net(nn.Module):
    def __init__(self,):
        super(wav_path_Net,self).__init__()
        
        self.model_wav2vec2 = torchaudio.models.wav2vec2_large_lv60k(aux_num_out=32)
        self.model_wav2vec2.load_state_dict(torch.load("../checkpoint/wav2vec2-large-lv60.pt"))
        #self.model_wav2vec2.aux = nn.Sequential(nn.Linear(1024,2,bias=True),)
        
        #input_dim =101951
        
        self.padded_frame=318
        
        self.fc = nn.Sequential(
                                nn.Linear(512,256),
                                nn.BatchNorm1d(256),
                                nn.ReLU(),
                                nn.Dropout(0.7),
                                nn.Linear(256,50),
                                nn.BatchNorm1d(50),
                                nn.ReLU(),
                                nn.Dropout(0.7),
                                nn.Linear(50,2),)
        
        
    def merge(self,hidden_feature):
        hidden_feature=hidden_feature.mean(axis=1)
        return hidden_feature
    
    def zero_padding(self,hidden_feature,seq_len):
        
        zero_padding_array=np.ones((len(seq_len),self.padded_frame, hidden_feature.size()[-1] ))
        for i in range(len(seq_len)):
            zero_padding_array[i:,seq_len[i]:,:]=0
            
        hidden_feature=hidden_feature*torch.from_numpy(zero_padding_array).float().to(DEVICE) # 318 * 2
        return hidden_feature
        
    def forward(self,x,sig_size):
        with torch.inference_mode():
            encoder_x,encoder_len = self.model_wav2vec2.feature_extractor(x,sig_size)
        x = encoder_x
        #x = self.zero_padding(x,encoder_len)
        x = self.merge(x)
        #x = torch.stack(x,dim=1)
        #x = x.view(-1,1024)
        x= self.fc(x)
    
        return x 
    


In [51]:
#7. Optimizer, Objective Function
def model_initialize(num):
    model = wav_path_Net(num).to(DEVICE)
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model

In [71]:
from torchinfo import summary

In [74]:
# get the model summary
from torchinfo import summary
#model=model_initialize(1)
#summary(model)

Layer (type:depth-idx)                                       Param #
wav_path_Net                                                 --
├─Wav2Vec2Model: 1-1                                         --
│    └─FeatureExtractor: 2-1                                 --
│    │    └─ModuleList: 3-1                                  4,210,176
│    └─Encoder: 2-2                                          --
│    │    └─FeatureProjection: 3-2                           526,336
│    │    └─Transformer: 3-3                                 310,701,184
│    └─Linear: 2-3                                           32,800
├─Sequential: 1-2                                            --
│    └─Linear: 2-4                                           51,250
│    └─BatchNorm1d: 2-5                                      100
│    └─ReLU: 2-6                                             --
│    └─Dropout: 2-7                                          --
│    └─Linear: 2-8                                           102
Tota

In [26]:
#7. Optimizer, Objective Function
def model_initialize():
    model = wav_path_Net().to(DEVICE)
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model

In [14]:
#8. 학습
def train(model,train_loader,optimizer, log_interval):
    model.train()
    correct = 0
    train_loss = 0
    for batch_idx,(image,label,sig_len) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        sig_len = sig_len.to(DEVICE)
        #데이터들 장비에 할당
        optimizer.zero_grad() # device 에 저장된 gradient 제거
        output = model(image,sig_len) # model로 output을 계산
        loss = criterion(output, label) #loss 계산
        train_loss += loss.item()
        prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
        correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
        loss.backward() # loss 값을 이용해 gradient를 계산
        optimizer.step() # Gradient 값을 이용해 파라미터 업데이트.
    train_loss/=len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss,train_accuracy


In [15]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def evaluate(model,valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label,sig_len in valid_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            sig_len = sig_len.to(DEVICE)
            output = model(image,sig_len)
            valid_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
            
            #true.false값을 sum해줌. item
        valid_loss /= len(valid_loader.dataset)
        valid_accuracy = 100. * correct / len(valid_loader.dataset)
        return valid_loss,valid_accuracy


In [16]:
#데이터 로더 제작 함수

def load_data(data_ind):

    train_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_train_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=True
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,
                                               ) # 순서가 암기되는것을 막기위해.

    validation_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_valid_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=False
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 
    return train_loader,validation_loader



In [17]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def test_evaluate(model,test_loader):
    model.eval()
    predictions = []
    answers = []
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label,sig_len in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            sig_len = sig_len.to(DEVICE)
            output = model(image,sig_len)
            output = F.softmax(output, dim=1).data.squeeze() # softmax 적용 (모델을 통과는 했지만, criterion는 안통과함.)
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            answers +=label
            predictions +=prediction
        return predictions,answers

In [32]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score



def test_grid():
    cf = np.zeros((2,2))
    cf_list = []

    
    
    for grid_num in range(0,24):
        
        average_accuracy = 0
        average_fscore = 0
        for data_ind in range(1,6):
            model=model_initialize(grid_num)
            check_path = '../checkpoint/checkpoint_w2v_large_model60k_'+str(grid_num)+'-'+str(data_ind)+'.pt'
            model.load_state_dict(torch.load(check_path))

            predictions,answers = test_evaluate(model, test_loader)
            predictions=[ dat.cpu().numpy() for dat in predictions]
            answers=[ dat.cpu().numpy() for dat in answers]


            cf = confusion_matrix(answers, predictions)
            cf_list.append(cf)

            acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
            average_accuracy+=acc
            precision=cf[0,0]/(cf[0,0]+cf[1,0])
            recall=cf[0,0]/(cf[0,0]+cf[0,1])
            #fscore=2*precision*recall/(precision+recall)

            #fscroe macro추가
            fscore = f1_score(answers,predictions,average='macro')
            average_fscore+=fscore
            

            print('{}번 모델 / {}번 fold'.format(grid_num+1,data_ind))
            print("Accuracy : {:.4f}% ".format(acc*100))
            print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
            print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
            print("f score : {:.4f} ".format(fscore))
            print(cf)
            print("-----")
        print("----")
        print("* {}번 모델 {:.4f}% / score: {:.4f}".format(grid_num+1, 100*average_accuracy/5,average_fscore/5))
        print("----")


In [31]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score



def test_grid():
    cf = np.zeros((2,2))
    cf_list = []

    for data_ind in range(1,2):
        model=model_initialize(grid_num)
        check_path = '../checkpoint/checkpoint_w2v_large_model60k_encoder'+ '-' +str(data_ind)+'.pt'
        model.load_state_dict(torch.load(check_path))

        predictions,answers = test_evaluate(model, test_loader)
        predictions=[ dat.cpu().numpy() for dat in predictions]
        answers=[ dat.cpu().numpy() for dat in answers]


        cf = confusion_matrix(answers, predictions)
        cf_list.append(cf)

        acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
        precision=cf[0,0]/(cf[0,0]+cf[1,0])
        recall=cf[0,0]/(cf[0,0]+cf[0,1])
        #fscore=2*precision*recall/(precision+recall)

        #fscroe macro추가
        fscore = f1_score(answers,predictions,average='macro')

        print('{}번 모델'.format(grid_num+1))
        print("Accuracy : {:.4f}% ".format(acc*100))
        print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
        print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
        print("f score : {:.4f} ".format(fscore))
        print(cf)
        print("-----")




In [28]:
#10. 학습 및 평가.
# kfold 적용
train_accs = []
valid_accs = []
for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_w2v_large_model60k_encoder'+ '-' +str(data_ind)+'.pt'
    print(check_path)
    early_stopping = EarlyStopping(patience = 5, verbose = True, path=check_path)
    train_loader,validation_loader = load_data(data_ind-1)

    best_train_acc=0 # accuracy 기록용
    best_valid_acc=0

    model=model_initialize()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)


    print("[{} 교차검증] 학습 시작\n ----- ".format(data_ind))
    for Epoch in range(1,EPOCHS+1):
        train_loss,train_accuracy=train(model,train_loader,optimizer,log_interval=31)
        valid_loss,valid_accuracy = evaluate(model, validation_loader)


        print("\n[EPOCH:{}]\t Train Loss:{:.4f}\t Train Acc:{:.2f} %  | \tValid Loss:{:.4f} \tValid Acc: {:.2f} %\n".
              format(Epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))


        early_stopping(valid_loss, model)
        if -early_stopping.best_score == valid_loss:
            best_train_acc, best_valid_acc = train_accuracy,valid_accuracy

        if early_stopping.early_stop:
                train_accs.append(best_train_acc)
                valid_accs.append(best_valid_acc)
                print("[{} 교차검증] Early stopping".format(data_ind))
                break

        if Epoch==EPOCHS:
            #만약 early stop 없이 40 epoch라서 중지 된 경우.
            train_accs.append(best_train_acc)
            valid_accs.append(best_valid_acc)

../checkpoint/checkpoint_w2v_large_model60k_encoder-1.pt


TypeError: __init__() missing 1 required positional argument: 'num'

In [19]:
#10. 학습 및 평가.
# kfold 적용

all_train_accs =[]
all_valid_accs =[]

for num in range(24):
    print("{}-번 레이어".format(num))
    train_accs = []
    valid_accs = []

    for data_ind in range(1,6):

        check_path = '../checkpoint/checkpoint_w2v_large_model60k_'+str(num)+'-'+str(data_ind)+'.pt'
        print(check_path)
        early_stopping = EarlyStopping(patience = 5, verbose = True, path=check_path)
        train_loader,validation_loader = load_data(data_ind-1)

        best_train_acc=0 # accuracy 기록용
        best_valid_acc=0

        model=model_initialize(num)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),lr=0.001)


        print("[{} 교차검증] 학습 시작\n ----- ".format(data_ind))
        for Epoch in range(1,EPOCHS+1):
            train_loss,train_accuracy=train(model,train_loader,optimizer,log_interval=31)
            valid_loss,valid_accuracy = evaluate(model, validation_loader)


            print("\n[EPOCH:{}]\t Train Loss:{:.4f}\t Train Acc:{:.2f} %  | \tValid Loss:{:.4f} \tValid Acc: {:.2f} %\n".
                  format(Epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))
            

            early_stopping(valid_loss, model)
            if -early_stopping.best_score == valid_loss:
                best_train_acc, best_valid_acc = train_accuracy,valid_accuracy

            if early_stopping.early_stop:
                    train_accs.append(best_train_acc)
                    valid_accs.append(best_valid_acc)
                    print("[{} 교차검증] Early stopping".format(data_ind))
                    break

            if Epoch==EPOCHS:
                #만약 early stop 없이 40 epoch라서 중지 된 경우.
                train_accs.append(best_train_acc)
                valid_accs.append(best_valid_acc)
    all_train_accs.append(train_accs)
    all_valid_accs.append(valid_accs)

0-번 레이어
../checkpoint/checkpoint_w2v_large_model60k_0-1.pt
[1 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0162	 Train Acc:75.74 %  | 	Valid Loss:0.0146 	Valid Acc: 79.70 %

Validation loss decreased (inf --> 0.014616).  Saving model ...

[EPOCH:2]	 Train Loss:0.0110	 Train Acc:86.21 %  | 	Valid Loss:0.0141 	Valid Acc: 82.23 %

Validation loss decreased (0.014616 --> 0.014112).  Saving model ...

[EPOCH:3]	 Train Loss:0.0105	 Train Acc:88.05 %  | 	Valid Loss:0.0165 	Valid Acc: 84.26 %

EarlyStopping counter: 1 out of 5

[EPOCH:4]	 Train Loss:0.0098	 Train Acc:88.67 %  | 	Valid Loss:0.0144 	Valid Acc: 80.71 %

EarlyStopping counter: 2 out of 5

[EPOCH:5]	 Train Loss:0.0087	 Train Acc:89.29 %  | 	Valid Loss:0.0127 	Valid Acc: 85.79 %

Validation loss decreased (0.014112 --> 0.012656).  Saving model ...

[EPOCH:6]	 Train Loss:0.0092	 Train Acc:87.81 %  | 	Valid Loss:0.0127 	Valid Acc: 83.25 %

EarlyStopping counter: 1 out of 5

[EPOCH:7]	 Train Loss:0.0082	 Train Acc:89.53 %  | 	Valid Los

[1 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0151	 Train Acc:77.83 %  | 	Valid Loss:0.0141 	Valid Acc: 81.73 %

Validation loss decreased (inf --> 0.014068).  Saving model ...

[EPOCH:2]	 Train Loss:0.0113	 Train Acc:85.47 %  | 	Valid Loss:0.0117 	Valid Acc: 84.26 %

Validation loss decreased (0.014068 --> 0.011698).  Saving model ...

[EPOCH:3]	 Train Loss:0.0101	 Train Acc:86.95 %  | 	Valid Loss:0.0119 	Valid Acc: 85.28 %

EarlyStopping counter: 1 out of 5

[EPOCH:4]	 Train Loss:0.0088	 Train Acc:89.90 %  | 	Valid Loss:0.0137 	Valid Acc: 84.77 %

EarlyStopping counter: 2 out of 5

[EPOCH:5]	 Train Loss:0.0085	 Train Acc:89.53 %  | 	Valid Loss:0.0118 	Valid Acc: 84.77 %

EarlyStopping counter: 3 out of 5

[EPOCH:6]	 Train Loss:0.0087	 Train Acc:88.42 %  | 	Valid Loss:0.0143 	Valid Acc: 83.76 %

EarlyStopping counter: 4 out of 5

[EPOCH:7]	 Train Loss:0.0086	 Train Acc:88.55 %  | 	Valid Loss:0.0118 	Valid Acc: 85.79 %

EarlyStopping counter: 5 out of 5
[1 교차검증] Early stopping
../che

[2 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0154	 Train Acc:75.62 %  | 	Valid Loss:0.0111 	Valid Acc: 82.74 %

Validation loss decreased (inf --> 0.011134).  Saving model ...

[EPOCH:2]	 Train Loss:0.0113	 Train Acc:86.21 %  | 	Valid Loss:0.0123 	Valid Acc: 82.23 %

EarlyStopping counter: 1 out of 5

[EPOCH:3]	 Train Loss:0.0100	 Train Acc:88.55 %  | 	Valid Loss:0.0117 	Valid Acc: 83.76 %

EarlyStopping counter: 2 out of 5

[EPOCH:4]	 Train Loss:0.0091	 Train Acc:89.29 %  | 	Valid Loss:0.0092 	Valid Acc: 85.79 %

Validation loss decreased (0.011134 --> 0.009247).  Saving model ...

[EPOCH:5]	 Train Loss:0.0087	 Train Acc:89.90 %  | 	Valid Loss:0.0108 	Valid Acc: 87.82 %

EarlyStopping counter: 1 out of 5

[EPOCH:6]	 Train Loss:0.0080	 Train Acc:91.26 %  | 	Valid Loss:0.0111 	Valid Acc: 85.28 %

EarlyStopping counter: 2 out of 5

[EPOCH:7]	 Train Loss:0.0075	 Train Acc:91.38 %  | 	Valid Loss:0.0114 	Valid Acc: 85.28 %

EarlyStopping counter: 3 out of 5

[EPOCH:8]	 Train Loss:0.0066	


[EPOCH:8]	 Train Loss:0.0073	 Train Acc:90.89 %  | 	Valid Loss:0.0185 	Valid Acc: 87.76 %

EarlyStopping counter: 5 out of 5
[5 교차검증] Early stopping
3-번 레이어
../checkpoint/checkpoint_w2v_large_model60k_3-1.pt
[1 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0143	 Train Acc:78.45 %  | 	Valid Loss:0.0149 	Valid Acc: 83.76 %

Validation loss decreased (inf --> 0.014930).  Saving model ...

[EPOCH:2]	 Train Loss:0.0105	 Train Acc:87.19 %  | 	Valid Loss:0.0117 	Valid Acc: 85.28 %

Validation loss decreased (0.014930 --> 0.011677).  Saving model ...

[EPOCH:3]	 Train Loss:0.0097	 Train Acc:89.04 %  | 	Valid Loss:0.0116 	Valid Acc: 84.77 %

Validation loss decreased (0.011677 --> 0.011624).  Saving model ...

[EPOCH:4]	 Train Loss:0.0083	 Train Acc:90.89 %  | 	Valid Loss:0.0117 	Valid Acc: 83.76 %

EarlyStopping counter: 1 out of 5

[EPOCH:5]	 Train Loss:0.0076	 Train Acc:90.76 %  | 	Valid Loss:0.0105 	Valid Acc: 86.29 %

Validation loss decreased (0.011624 --> 0.010537).  Saving model ...

[E


[EPOCH:10]	 Train Loss:0.0071	 Train Acc:91.11 %  | 	Valid Loss:0.0117 	Valid Acc: 89.85 %

EarlyStopping counter: 1 out of 5

[EPOCH:11]	 Train Loss:0.0075	 Train Acc:90.49 %  | 	Valid Loss:0.0086 	Valid Acc: 91.37 %

EarlyStopping counter: 2 out of 5

[EPOCH:12]	 Train Loss:0.0057	 Train Acc:93.95 %  | 	Valid Loss:0.0082 	Valid Acc: 90.36 %

Validation loss decreased (0.008230 --> 0.008214).  Saving model ...

[EPOCH:13]	 Train Loss:0.0060	 Train Acc:92.96 %  | 	Valid Loss:0.0092 	Valid Acc: 89.85 %

EarlyStopping counter: 1 out of 5

[EPOCH:14]	 Train Loss:0.0054	 Train Acc:93.95 %  | 	Valid Loss:0.0088 	Valid Acc: 89.34 %

EarlyStopping counter: 2 out of 5

[EPOCH:15]	 Train Loss:0.0056	 Train Acc:93.46 %  | 	Valid Loss:0.0095 	Valid Acc: 90.36 %

EarlyStopping counter: 3 out of 5

[EPOCH:16]	 Train Loss:0.0051	 Train Acc:93.70 %  | 	Valid Loss:0.0097 	Valid Acc: 89.34 %

EarlyStopping counter: 4 out of 5

[EPOCH:17]	 Train Loss:0.0051	 Train Acc:94.57 %  | 	Valid Loss:0.0113 	Val

...


[EPOCH:5]	 Train Loss:0.0095	 Train Acc:89.38 %  | 	Valid Loss:0.0099 	Valid Acc: 86.80 %

Validation loss decreased (0.011233 --> 0.009901).  Saving model ...

[EPOCH:6]	 Train Loss:0.0090	 Train Acc:89.14 %  | 	Valid Loss:0.0098 	Valid Acc: 86.29 %

Validation loss decreased (0.009901 --> 0.009834).  Saving model ...

[EPOCH:7]	 Train Loss:0.0081	 Train Acc:88.89 %  | 	Valid Loss:0.0121 	Valid Acc: 84.26 %

EarlyStopping counter: 1 out of 5

[EPOCH:8]	 Train Loss:0.0090	 Train Acc:87.65 %  | 	Valid Loss:0.0115 	Valid Acc: 85.28 %

EarlyStopping counter: 2 out of 5

[EPOCH:9]	 Train Loss:0.0076	 Train Acc:91.11 %  | 	Valid Loss:0.0118 	Valid Acc: 86.29 %

EarlyStopping counter: 3 out of 5

[EPOCH:10]	 Train Loss:0.0079	 Train Acc:89.63 %  | 	Valid Loss:0.0125 	Valid Acc: 84.77 %

EarlyStopping counter: 4 out of 5

[EPOCH:11]	 Train Loss:0.0084	 Train Acc:89.51 %  | 	Valid Loss:0.0093 	Valid Acc: 86.80 %

Validation loss decreased (0.009834 --> 0.009330).  Saving model ...

[EPOCH:12]


[EPOCH:16]	 Train Loss:0.0072	 Train Acc:90.52 %  | 	Valid Loss:0.0143 	Valid Acc: 84.26 %

EarlyStopping counter: 5 out of 5
[1 교차검증] Early stopping
../checkpoint/checkpoint_w2v_large_model60k_20-2.pt
[2 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0161	 Train Acc:76.23 %  | 	Valid Loss:0.0136 	Valid Acc: 77.66 %

Validation loss decreased (inf --> 0.013581).  Saving model ...

[EPOCH:2]	 Train Loss:0.0119	 Train Acc:85.59 %  | 	Valid Loss:0.0119 	Valid Acc: 80.71 %

Validation loss decreased (0.013581 --> 0.011901).  Saving model ...

[EPOCH:3]	 Train Loss:0.0103	 Train Acc:89.04 %  | 	Valid Loss:0.0117 	Valid Acc: 84.77 %

Validation loss decreased (0.011901 --> 0.011747).  Saving model ...

[EPOCH:4]	 Train Loss:0.0101	 Train Acc:87.93 %  | 	Valid Loss:0.0148 	Valid Acc: 83.25 %

EarlyStopping counter: 1 out of 5

[EPOCH:5]	 Train Loss:0.0090	 Train Acc:88.30 %  | 	Valid Loss:0.0112 	Valid Acc: 82.74 %

Validation loss decreased (0.011747 --> 0.011165).  Saving model ...

[EPOCH:6


[EPOCH:10]	 Train Loss:0.0082	 Train Acc:89.90 %  | 	Valid Loss:0.0106 	Valid Acc: 88.27 %

EarlyStopping counter: 5 out of 5
[5 교차검증] Early stopping
21-번 레이어
../checkpoint/checkpoint_w2v_large_model60k_21-1.pt
[1 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0226	 Train Acc:57.64 %  | 	Valid Loss:0.0312 	Valid Acc: 51.27 %

Validation loss decreased (inf --> 0.031182).  Saving model ...

[EPOCH:2]	 Train Loss:0.0224	 Train Acc:58.99 %  | 	Valid Loss:0.0767 	Valid Acc: 48.73 %

EarlyStopping counter: 1 out of 5

[EPOCH:3]	 Train Loss:0.0214	 Train Acc:59.11 %  | 	Valid Loss:0.0864 	Valid Acc: 48.73 %

EarlyStopping counter: 2 out of 5

[EPOCH:4]	 Train Loss:0.0212	 Train Acc:63.79 %  | 	Valid Loss:0.0436 	Valid Acc: 48.73 %

EarlyStopping counter: 3 out of 5

[EPOCH:5]	 Train Loss:0.0212	 Train Acc:65.02 %  | 	Valid Loss:0.0362 	Valid Acc: 48.73 %

EarlyStopping counter: 4 out of 5

[EPOCH:6]	 Train Loss:0.0203	 Train Acc:65.64 %  | 	Valid Loss:0.0215 	Valid Acc: 75.13 %

Validation lo


[EPOCH:5]	 Train Loss:0.0231	 Train Acc:55.91 %  | 	Valid Loss:0.0355 	Valid Acc: 49.75 %

EarlyStopping counter: 3 out of 5

[EPOCH:6]	 Train Loss:0.0225	 Train Acc:56.90 %  | 	Valid Loss:0.0247 	Valid Acc: 53.30 %

Validation loss decreased (0.025319 --> 0.024651).  Saving model ...

[EPOCH:7]	 Train Loss:0.0224	 Train Acc:59.61 %  | 	Valid Loss:0.0831 	Valid Acc: 48.73 %

EarlyStopping counter: 1 out of 5

[EPOCH:8]	 Train Loss:0.0212	 Train Acc:60.71 %  | 	Valid Loss:0.0239 	Valid Acc: 56.85 %

Validation loss decreased (0.024651 --> 0.023942).  Saving model ...

[EPOCH:9]	 Train Loss:0.0213	 Train Acc:60.59 %  | 	Valid Loss:0.0270 	Valid Acc: 55.84 %

EarlyStopping counter: 1 out of 5

[EPOCH:10]	 Train Loss:0.0210	 Train Acc:61.45 %  | 	Valid Loss:0.0401 	Valid Acc: 48.73 %

EarlyStopping counter: 2 out of 5

[EPOCH:11]	 Train Loss:0.0215	 Train Acc:59.73 %  | 	Valid Loss:0.0256 	Valid Acc: 51.27 %

EarlyStopping counter: 3 out of 5

[EPOCH:12]	 Train Loss:0.0207	 Train Acc:63.5


[EPOCH:4]	 Train Loss:0.0227	 Train Acc:54.68 %  | 	Valid Loss:0.0230 	Valid Acc: 64.47 %

Validation loss decreased (0.023031 --> 0.022990).  Saving model ...

[EPOCH:5]	 Train Loss:0.0226	 Train Acc:56.53 %  | 	Valid Loss:0.0523 	Valid Acc: 48.73 %

EarlyStopping counter: 1 out of 5

[EPOCH:6]	 Train Loss:0.0220	 Train Acc:54.93 %  | 	Valid Loss:0.0234 	Valid Acc: 57.36 %

EarlyStopping counter: 2 out of 5

[EPOCH:7]	 Train Loss:0.0218	 Train Acc:56.77 %  | 	Valid Loss:0.0243 	Valid Acc: 54.31 %

EarlyStopping counter: 3 out of 5

[EPOCH:8]	 Train Loss:0.0217	 Train Acc:57.88 %  | 	Valid Loss:0.0236 	Valid Acc: 56.35 %

EarlyStopping counter: 4 out of 5

[EPOCH:9]	 Train Loss:0.0216	 Train Acc:60.47 %  | 	Valid Loss:0.0324 	Valid Acc: 48.73 %

EarlyStopping counter: 5 out of 5
[2 교차검증] Early stopping
../checkpoint/checkpoint_w2v_large_model60k_23-3.pt
[3 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0250	 Train Acc:50.74 %  | 	Valid Loss:0.0438 	Valid Acc: 51.78 %

Validation loss de

In [31]:
for num, valid_fold in enumerate(all_valid_accs):
    print("-----\n")
    avg_train = []
    avg_valid = []
    for num_fold,valid in enumerate(valid_fold):
        print("{}번 모델. {} 폴드.  train :{:.4f}/ valid: {:.4f}".format(num+1,num_fold+1,all_train_accs[num][num_fold],valid) )
        avg_train.append(all_train_accs[num][num_fold])
        avg_valid.append(valid)
    print("* 평균 train acc : {:.4f}% / 평균 valid acc : {:.4f}% ".format(np.array(avg_train).mean(), np.array(avg_valid).mean()) )

-----

1번 모델. 1 폴드.  train :89.5320/ valid: 82.2335
1번 모델. 2 폴드.  train :87.8079/ valid: 86.8020
1번 모델. 3 폴드.  train :87.6543/ valid: 86.2944
1번 모델. 4 폴드.  train :88.5185/ valid: 91.3706
1번 모델. 5 폴드.  train :86.6995/ valid: 87.7551
* 평균 train acc : 88.0424% / 평균 valid acc : 86.8911% 
-----

2번 모델. 1 폴드.  train :85.4680/ valid: 84.2640
2번 모델. 2 폴드.  train :87.1921/ valid: 88.8325
2번 모델. 3 폴드.  train :90.6173/ valid: 89.3401
2번 모델. 4 폴드.  train :84.8148/ valid: 89.8477
2번 모델. 5 폴드.  train :86.8227/ valid: 88.7755
* 평균 train acc : 86.9830% / 평균 valid acc : 88.2120% 
-----

3번 모델. 1 폴드.  train :88.6700/ valid: 85.7868
3번 모델. 2 폴드.  train :89.2857/ valid: 85.7868
3번 모델. 3 폴드.  train :94.5679/ valid: 89.3401
3번 모델. 4 폴드.  train :93.0864/ valid: 91.8782
3번 모델. 5 폴드.  train :88.7931/ valid: 87.7551
* 평균 train acc : 90.8806% / 평균 valid acc : 88.1094% 
-----

4번 모델. 1 폴드.  train :90.7635/ valid: 86.2944
4번 모델. 2 폴드.  train :93.1034/ valid: 87.8173
4번 모델. 3 폴드.  train :93.7037/ valid: 90.3553
4번 

In [21]:
for num,valid in enumerate(valid_accs):
    print("{}번 모델. train :{:.4f}/ valid: {:.4f}".format(num+1,train_accs[num],valid) )

1번 모델. train :89.6552/ valid: 84.7716
2번 모델. train :91.1330/ valid: 84.2640
3번 모델. train :92.8571/ valid: 85.7868
4번 모델. train :91.8719/ valid: 85.7868
5번 모델. train :86.6995/ valid: 85.7868
6번 모델. train :90.2709/ valid: 86.8020
7번 모델. train :92.8571/ valid: 87.8173
8번 모델. train :90.5172/ valid: 87.8173
9번 모델. train :93.8424/ valid: 86.2944
10번 모델. train :91.6256/ valid: 87.8173
11번 모델. train :90.6404/ valid: 87.8173
12번 모델. train :90.1478/ valid: 85.7868
13번 모델. train :85.9606/ valid: 85.2792
14번 모델. train :89.5320/ valid: 85.2792
15번 모델. train :90.6404/ valid: 86.2944
16번 모델. train :90.5172/ valid: 87.3096
17번 모델. train :88.0542/ valid: 87.3096
18번 모델. train :88.1773/ valid: 88.3249
19번 모델. train :88.4236/ valid: 85.7868
20번 모델. train :90.7635/ valid: 86.2944
21번 모델. train :91.9951/ valid: 85.2792
22번 모델. train :64.4089/ valid: 68.0203
23번 모델. train :56.7734/ valid: 51.7766
24번 모델. train :63.6700/ valid: 59.8985


In [22]:
pd.DataFrame({'train':train_accs,'vaild':valid_accs},).to_csv("./result_transformer_organics.csv")

In [ ]:
pd.DataFrame({'train':train_accs,'vaild':valid_accs},).to_csv("./result_encoder.csv")

In [33]:
#K FOLD
test_grid()

1번 모델 / 1번 fold
Accuracy : 86.2348% 
Precision (pathology 예측한 것중 맞는 것) : 0.8981
recall (실제 pathology 중  예측이 맞는 것) : 0.8083
f score : 0.8615 
[[ 97  23]
 [ 11 116]]
-----
1번 모델 / 2번 fold
Accuracy : 86.6397% 
Precision (pathology 예측한 것중 맞는 것) : 0.9065
recall (실제 pathology 중  예측이 맞는 것) : 0.8083
f score : 0.8655 
[[ 97  23]
 [ 10 117]]
-----
1번 모델 / 3번 fold
Accuracy : 87.0445% 
Precision (pathology 예측한 것중 맞는 것) : 0.8929
recall (실제 pathology 중  예측이 맞는 것) : 0.8333
f score : 0.8700 
[[100  20]
 [ 12 115]]
-----
1번 모델 / 4번 fold
Accuracy : 87.8543% 
Precision (pathology 예측한 것중 맞는 것) : 0.8629
recall (실제 pathology 중  예측이 맞는 것) : 0.8917
f score : 0.8785 
[[107  13]
 [ 17 110]]
-----
1번 모델 / 5번 fold
Accuracy : 87.8543% 
Precision (pathology 예측한 것중 맞는 것) : 0.8879
recall (실제 pathology 중  예측이 맞는 것) : 0.8583
f score : 0.8783 
[[103  17]
 [ 13 114]]
-----
----
* 1번 모델 87.1255% / score: 0.8708
----
2번 모델 / 1번 fold
Accuracy : 87.0445% 
Precision (pathology 예측한 것중 맞는 것) : 0.8729
recall (실제 pathology 중  예측이

10번 모델 / 2번 fold
Accuracy : 88.6640% 
Precision (pathology 예측한 것중 맞는 것) : 0.8898
recall (실제 pathology 중  예측이 맞는 것) : 0.8750
f score : 0.8865 
[[105  15]
 [ 13 114]]
-----
10번 모델 / 3번 fold
Accuracy : 89.8785% 
Precision (pathology 예측한 것중 맞는 것) : 0.9279
recall (실제 pathology 중  예측이 맞는 것) : 0.8583
f score : 0.8984 
[[103  17]
 [  8 119]]
-----
10번 모델 / 4번 fold
Accuracy : 87.4494% 
Precision (pathology 예측한 것중 맞는 것) : 0.9159
recall (실제 pathology 중  예측이 맞는 것) : 0.8167
f score : 0.8737 
[[ 98  22]
 [  9 118]]
-----
10번 모델 / 5번 fold
Accuracy : 88.6640% 
Precision (pathology 예측한 것중 맞는 것) : 0.9107
recall (실제 pathology 중  예측이 맞는 것) : 0.8500
f score : 0.8862 
[[102  18]
 [ 10 117]]
-----
----
* 10번 모델 88.4211% / score: 0.8838
----
11번 모델 / 1번 fold
Accuracy : 89.0688% 
Precision (pathology 예측한 것중 맞는 것) : 0.9043
recall (실제 pathology 중  예측이 맞는 것) : 0.8667
f score : 0.8904 
[[104  16]
 [ 11 116]]
-----
11번 모델 / 2번 fold
Accuracy : 87.8543% 
Precision (pathology 예측한 것중 맞는 것) : 0.9167
recall (실제 pathology

19번 모델 / 3번 fold
Accuracy : 89.4737% 
Precision (pathology 예측한 것중 맞는 것) : 0.9434
recall (실제 pathology 중  예측이 맞는 것) : 0.8333
f score : 0.8940 
[[100  20]
 [  6 121]]
-----
19번 모델 / 4번 fold
Accuracy : 89.0688% 
Precision (pathology 예측한 것중 맞는 것) : 0.9697
recall (실제 pathology 중  예측이 맞는 것) : 0.8000
f score : 0.8893 
[[ 96  24]
 [  3 124]]
-----
19번 모델 / 5번 fold
Accuracy : 89.4737% 
Precision (pathology 예측한 것중 맞는 것) : 0.9273
recall (실제 pathology 중  예측이 맞는 것) : 0.8500
f score : 0.8942 
[[102  18]
 [  8 119]]
-----
----
* 19번 모델 89.2308% / score: 0.8916
----
20번 모델 / 1번 fold
Accuracy : 90.2834% 
Precision (pathology 예측한 것중 맞는 것) : 0.9211
recall (실제 pathology 중  예측이 맞는 것) : 0.8750
f score : 0.9026 
[[105  15]
 [  9 118]]
-----
20번 모델 / 2번 fold
Accuracy : 86.6397% 
Precision (pathology 예측한 것중 맞는 것) : 0.9065
recall (실제 pathology 중  예측이 맞는 것) : 0.8083
f score : 0.8655 
[[ 97  23]
 [ 10 117]]
-----
20번 모델 / 3번 fold
Accuracy : 90.2834% 
Precision (pathology 예측한 것중 맞는 것) : 0.9211
recall (실제 pathology

In [23]:
# 24 레이어
test_grid()

1번 모델
Accuracy : 86.6397% 
Precision (pathology 예측한 것중 맞는 것) : 0.8850
recall (실제 pathology 중  예측이 맞는 것) : 0.8333
f score : 0.8660 
[[100  20]
 [ 13 114]]
-----
2번 모델
Accuracy : 87.0445% 
Precision (pathology 예측한 것중 맞는 것) : 0.8929
recall (실제 pathology 중  예측이 맞는 것) : 0.8333
f score : 0.8700 
[[100  20]
 [ 12 115]]
-----
3번 모델
Accuracy : 87.4494% 
Precision (pathology 예측한 것중 맞는 것) : 0.9238
recall (실제 pathology 중  예측이 맞는 것) : 0.8083
f score : 0.8735 
[[ 97  23]
 [  8 119]]
-----
4번 모델
Accuracy : 89.0688% 
Precision (pathology 예측한 것중 맞는 것) : 0.9115
recall (실제 pathology 중  예측이 맞는 것) : 0.8583
f score : 0.8903 
[[103  17]
 [ 10 117]]
-----
5번 모델
Accuracy : 89.4737% 
Precision (pathology 예측한 것중 맞는 것) : 0.9273
recall (실제 pathology 중  예측이 맞는 것) : 0.8500
f score : 0.8942 
[[102  18]
 [  8 119]]
-----
6번 모델
Accuracy : 88.2591% 
Precision (pathology 예측한 것중 맞는 것) : 0.9174
recall (실제 pathology 중  예측이 맞는 것) : 0.8333
f score : 0.8820 
[[100  20]
 [  9 118]]
-----
7번 모델
Accuracy : 89.8785% 
Precision (pa

In [ ]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

cf = np.zeros((2,2))
cf_list = []
average_accuracy = 0
average_fscore = 0

for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_w2v_'+str(data_ind)+'.pt'
    model.load_state_dict(torch.load(check_path))

    predictions,answers = test_evaluate(model, test_loader)
    predictions=[ dat.cpu().numpy() for dat in predictions]
    answers=[ dat.cpu().numpy() for dat in answers]

    
    cf = confusion_matrix(answers, predictions)
    cf_list.append(cf)
    
    acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
    average_accuracy+=acc
    precision=cf[0,0]/(cf[0,0]+cf[1,0])
    recall=cf[0,0]/(cf[0,0]+cf[0,1])
    #fscore=2*precision*recall/(precision+recall)
    
    #fscroe macro추가
    fscore = f1_score(answers,predictions,average='macro')
    average_fscore+=fscore
    
    print('{}번 모델'.format(data_ind))
    print("Accuracy : {:.4f}% ".format(acc*100))
    print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
    print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
    print("f score : {:.4f} ".format(fscore))
    print(cf)
    print("-----")

print("평균 acc : {:.4f}".format(average_accuracy/5))
print("평균 f1score : {:.4f}".format(average_fscore/5))


